In [1]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd

In [2]:
# Load the dataset
hotels_df = pd.read_csv('https://raw.githubusercontent.com/kristxna/Datasets/refs/heads/main/hotels.csv')

In [3]:
hotels_df.columns

Index(['Room_ID', 'Floor', 'Square_Feet', 'Cleaning_Time_Hours'], dtype='object')

In [4]:
hotels_df.head()

,Room_ID,Floor,Square_Feet,Cleaning_Time_Hours
0,1,3,682,1.814545
1,2,9,223,0.562727
2,3,7,506,1.334545
3,4,1,561,1.484545
4,5,14,424,1.110909


In [5]:
print("\n=================== Original Optimization Model ===================")
# Part (a) to (e) build optimization model

# Extract relevant data
floors = hotels_df['Floor'].unique()
rooms = hotels_df['Room_ID'].unique()
num_attendants = 8
max_sqft_per_day = 3500
max_overtime_hours = 2
base_wage = 25
extra_floor_cost = 75

def get_room_data():
    room_data = {}
    for _, row in hotels_df.iterrows():
        floor, room, sqft, time = row['Floor'], row['Room_ID'], row['Square_Feet'], row['Cleaning_Time_Hours']
        if floor not in room_data:
            room_data[floor] = []
        room_data[floor].append((room, sqft, time))
    return room_data

room_data = get_room_data()

# Initialize model 
model = gp.Model("Hotel Cleaning Scheduling")

# Decision Variables
x = model.addVars(num_attendants, floors, rooms, vtype=GRB.BINARY, name="Assign")  # Whether attendant i cleans room j on floor k
f = model.addVars(num_attendants, floors, vtype=GRB.BINARY, name="FloorAssign")  # Whether attendant is assigned to a floor
overtime = model.addVars(num_attendants, vtype=GRB.INTEGER, lb=0, ub=max_overtime_hours, name="Overtime")  # Overtime hours
over_sqft = model.addVars(num_attendants, vtype=GRB.BINARY, name="OverSqft")  # If attendant cleans more than 3500 sqft

# Objective Function: Minimize cost
model.setObjective(
    gp.quicksum(base_wage * 8 + 1.5 * base_wage * overtime[i] + extra_floor_cost * gp.quicksum(f[i, k] for k in floors) 
                + 2 * base_wage * over_sqft[i] for i in range(num_attendants)), GRB.MINIMIZE)

# Constraints
# Ensure all rooms are assigned to one attendant
for k in floors:
    for j in room_data[k]:
        model.addConstr(gp.quicksum(x[i, k, j[0]] for i in range(num_attendants)) == 1, f"RoomAssignment_{k}_{j[0]}")

# Floor assignment consistency
for i in range(num_attendants):
    for k in floors:
        for j in room_data[k]:
            model.addConstr(x[i, k, j[0]] <= f[i, k], f"FloorConsistency_{i}_{k}_{j[0]}")

# Each attendant cleans between 2 to 4 floors
for i in range(num_attendants):
    model.addConstr(gp.quicksum(f[i, k] for k in floors) >= 2, f"MinFloors_{i}")
    model.addConstr(gp.quicksum(f[i, k] for k in floors) <= 4, f"MaxFloors_{i}")

# Ensure attendants do not exceed 3500 sqft before extra pay applies
for i in range(num_attendants):
    model.addConstr(gp.quicksum(x[i, k, j[0]] * j[1] for k in floors for j in room_data[k]) <= max_sqft_per_day + over_sqft[i] * 10000, f"SqFtLimit_{i}")

# Ensure attendants work within 8 hours with a maximum of 2 overtime hours
for i in range(num_attendants):
    model.addConstr(gp.quicksum(x[i, k, j[0]] * j[2] for k in floors for j in room_data[k]) <= 8 + overtime[i], f"WorkHours_{i}")


# Solve the model
model.optimize()

# Check that fik = 1 when at least one room is assigned on floor k
for i in range(num_attendants):
    for k in floors:
        assigned_rooms = [j[0] for j in room_data[k] if x[i, k, j[0]].x > 0.5]
        if assigned_rooms:  # Check if any rooms were assigned
            assert f[i, k].x == 1, f"Error: f[{i},{k}] should be 1 but is {f[i, k].x}"
print("Binary floor assignment constraint verified")

# Display results
if model.status == GRB.OPTIMAL:
    total_overtime = sum(overtime[i].x for i in range(num_attendants))
    total_floor_violations = sum(max(0, sum(f[i, k].x for k in floors) - 2) for i in range(num_attendants))
    print(f"Optimal Staffing Cost for the Day: {model.objVal}")
    print(f"Total Overtime Hours: {total_overtime}")
    print(f"Total Floor Violations: {total_floor_violations}")

    for i in range(num_attendants):
        assigned_floors = [k for k in floors if f[i, k].x > 0.5]
        num_floors = len(assigned_floors)
        total_sqft_cleaned = sum(x[i, k, j[0]].x * j[1] for k in floors for j in room_data[k])
        overtime_hours = max(0, overtime[i].x)

        # Wage Calculation
        base_pay = base_wage * 8  # Fixed daily pay
        overtime_pay = 1.5 * base_wage * overtime_hours  # Overtime pay
        extra_floor_pay = extra_floor_cost * max(0, num_floors - 2)  # Extra pay for more than 2 floors
        extra_sqft_pay = 2 * base_wage * (total_sqft_cleaned > max_sqft_per_day)  # Double pay if > 3500 sqft

        total_wage = base_pay + overtime_pay + extra_floor_pay + extra_sqft_pay  # Sum of all pay components

        formatted_floors = ", ".join(str(int(k)) for k in assigned_floors)
        print(f"Attendant {i+1}: Floors [{formatted_floors}], Overtime: {overtime_hours} hours, "
              f"Total Sqft Cleaned: {total_sqft_cleaned}, Total Wage: ${total_wage:.2f}")
else:
    print("No optimal solution found.")


=================== Original Optimization Model ===================
Set parameter Username
Set parameter LicenseID to value 2609994
Academic license - for non-commercial use only - expires 2026-01-14
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[x86] - Darwin 24.1.0 24B83)

CPU model: Intel(R) Core(TM) i5-8257U CPU @ 1.40GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 500 rows, 5952 columns and 2320 nonzeros
Model fingerprint: 0xb2dab0e2
Variable types: 0 continuous, 5952 integer (5944 binary)
Coefficient statistics:
  Matrix range     [6e-01, 1e+04]
  Objective range  [4e+01, 8e+01]
  Bounds range     [1e+00, 2e+00]
  RHS range        [1e+00, 4e+03]
Found heuristic solution: objective 4287.5000000
Presolve removed 0 rows and 5408 columns
Presolve time: 0.01s
Presolved: 500 rows, 544 columns, 2320 nonzeros
Variable types: 0 continuous, 544 integer (536 binary)

Root relaxation: objective 2.860511e+03, 577 iterations, 0.0

In [6]:
# (f) Model relaxation
print("\n=================== Relaxed Model ===================")
relaxed_model = model.relax()
relaxed_model.optimize()
if relaxed_model.status == GRB.OPTIMAL:
    print("(f) Relaxed Model Cost:", relaxed_model.objVal)


=================== Relaxed Model ===================
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[x86] - Darwin 24.1.0 24B83)

CPU model: Intel(R) Core(TM) i5-8257U CPU @ 1.40GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 500 rows, 5952 columns and 2320 nonzeros
Model fingerprint: 0xcce92f88
Coefficient statistics:
  Matrix range     [6e-01, 1e+04]
  Objective range  [4e+01, 8e+01]
  Bounds range     [1e+00, 2e+00]
  RHS range        [1e+00, 4e+03]
Presolve removed 8 rows and 5408 columns
Presolve time: 0.01s
Presolved: 492 rows, 552 columns, 2216 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.8605114e+03   0.000000e+00   0.000000e+00      0s

Use crossover to convert LP symmetric solution to basic solution...
Crossover log...

       0 DPushes remaining with DInf 0.0000000e+00                 0s

     468 PPushes remaining with PInf 0.0000000e+00                 0s
       0 PP

In [7]:
# Part (g) Convert binary to continuous and solve manually relaxed problem
print("\n=================== Manually Relaxed Model ===================")
for v in model.getVars():
    v.vtype = GRB.CONTINUOUS
model.optimize()
print(f"Manually Relaxed Solution Cost: {model.objVal}")


=================== Manually Relaxed Model ===================
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[x86] - Darwin 24.1.0 24B83)

CPU model: Intel(R) Core(TM) i5-8257U CPU @ 1.40GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 500 rows, 5952 columns and 2320 nonzeros
Model fingerprint: 0xcce92f88
Coefficient statistics:
  Matrix range     [6e-01, 1e+04]
  Objective range  [4e+01, 8e+01]
  Bounds range     [1e+00, 2e+00]
  RHS range        [1e+00, 4e+03]
Presolve removed 8 rows and 5408 columns
Presolve time: 0.01s
Presolved: 492 rows, 552 columns, 2216 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.8605114e+03   0.000000e+00   0.000000e+00      0s

Use crossover to convert LP symmetric solution to basic solution...
Crossover log...

       0 DPushes remaining with DInf 0.0000000e+00                 0s

     468 PPushes remaining with PInf 0.0000000e+00                 0s
  

In [8]:
print(f"Binary Model Cost: {model.objVal}")
print(f"Relaxed Model Cost: {relaxed_model.objVal}")

Binary Model Cost: 2860.511363636364
Relaxed Model Cost: 2860.511363636364


In [9]:
print(f"Before 2x Overtime: Cost = {model.objVal}, Overtime Hours = {total_overtime}, Floor Violations = {total_floor_violations}")

Before 2x Overtime: Cost = 2860.511363636364, Overtime Hours = 4.0, Floor Violations = 0


In [10]:
# Part (i) Adjust overtime multiplier to 2.0 and re-optimize
print("\n=================== Updated Model with 2x Overtime Pay ===================")

overtime_multiplier = 2.0
model.setObjective(
    gp.quicksum(base_wage * (8 + overtime_multiplier * overtime[i]) for i in range(num_attendants)) +
    gp.quicksum(extra_floor_cost * gp.quicksum(f[i, k] for k in floors) for i in range(num_attendants)) +
    gp.quicksum(base_wage * 8 * over_sqft[i] for i in range(num_attendants)),
    GRB.MINIMIZE
)
model.optimize()
print(f"Updated Overtime Multiplier (2x): {model.objVal}")


=================== Updated Model with 2x Overtime Pay ===================
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[x86] - Darwin 24.1.0 24B83)

CPU model: Intel(R) Core(TM) i5-8257U CPU @ 1.40GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 500 rows, 5952 columns and 2320 nonzeros
Coefficient statistics:
  Matrix range     [6e-01, 1e+04]
  Objective range  [5e+01, 2e+02]
  Bounds range     [1e+00, 2e+00]
  RHS range        [1e+00, 4e+03]
LP warm-start: use basis

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.8806818e+03   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.02 seconds (0.00 work units)
Optimal objective  2.880681818e+03
Updated Overtime Multiplier (2x): 2880.681818181819


In [11]:
print(f"After 2x Overtime: Cost = {model.objVal}")

After 2x Overtime: Cost = 2880.681818181819
